In [626]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException


5성급 호텔 리뷰 웹크롤링

In [668]:
# 크롬 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [661]:
# 아고다 접속
url = 'https://www.agoda.com/ko-kr/'
driver.get(url)

In [440]:
# 광고 제거
driver.find_element('xpath','/html/body/div[12]/div[2]/div[3]/button[2]').click()

In [448]:
# 검색창 클릭
driver.find_element('xpath','//*[@id="textInput"]').click()

In [449]:
# 서울 입력
driver.find_element('xpath','//*[@id="textInput"]').send_keys('서울')

In [450]:
# 서울 클릭
driver.find_element('xpath','//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/ul/li[1]').click()

In [451]:
# 날짜 자동입력

actions = ActionChains(driver)

actions.send_keys(Keys.SPACE).perform()
actions.send_keys(Keys.SPACE).perform()

In [453]:
# 검색하기
driver.find_element('xpath','//*[@id="SearchBoxContainer"]/div[2]/div/button').click()

In [454]:
# 탭 이동
driver.switch_to.window(driver.window_handles[0])

# 탭 닫기
driver.close()

# 탭 바꾸기
driver.switch_to.window(driver.window_handles[0])

In [461]:
# 호텔 체크
driver.find_element(By.CSS_SELECTOR,'#SideBarLocationFilters > div:nth-child(2) > div.filter-items > ul > li:nth-child(1) > span > span:nth-child(1) > span').click()

In [462]:
# 5성급 체크
driver.find_element(By.CSS_SELECTOR,'#SideBarLocationFilters > div:nth-child(3) > div.filter-items > ul > li:nth-child(1) > span > span:nth-child(1) > span').click()

In [463]:
# 스크롤 다운
actions = ActionChains(driver)
last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    for _ in range (15):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(1)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height


In [ ]:
# 호텔명 가져오기
names = []

name_elements = driver.find_elements(By.CSS_SELECTOR, '.sc-jrAGrp.sc-kEjbxe.eDlaBj.dscgss')

for element in name_elements:
    text = element.text
    names.append(text)

len(names)

In [ ]:
#호텔 url 가져오기
urls = []

url_elements = driver.find_elements(By.CLASS_NAME, 'PropertyCard__Link')
for element in url_elements:
    url = element.get_attribute('href')
    urls.append(url)
len(urls)

In [631]:
df = pd.DataFrame({
    'name':names,
    'url':urls,
    'address':None,
    'rating':None,
    'review_rating':None,
    'review_star' : None,
    'review':None
})

In [550]:

# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [559]:
driver.find_element(By.CSS_SELECTOR,'#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select').click()

In [560]:
# 언어 선택 드롭다운 찾기
language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))

# "한국어" 선택
language_dropdown.select_by_visible_text("한국어")

In [ ]:
# 주소, 별점, 리뷰 가져오기

address = []
scores = []
reviews = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(6)
    
    # 주소 가져오기
    try:
        address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
    except ElementClickInterceptedException:
        continue
    
    # 이용후기 누르기
    try:
        driver.find_element(By.XPATH, '//*[@id="hotelNavBar"]/div/ul/li[4]/button/span').click()
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span/i[2]').click() #21번 전까지
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[2]').click() # 21번 전까지
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # 홈페이지 총점 가져오기
    try :
        stars.append(float(driver.find_element('xpath','//*[@id="property-critical-root"]/div/div[5]/div[2]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/p[1]').text))
    except NoSuchElementException:
        stars.append('')
    
    
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
    
    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
    
    # 다음 호텔로 넘어가기
    continue

In [656]:
len(df.iloc[0]['review_star'])

420

In [653]:
len(df.iloc[0]['review'])

420

In [648]:
df.iloc[0]['url']

'https://www.agoda.com/ko-kr/grand-hilton-seoul/hotel/seoul-kr.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-12-29&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,8&los=1&searchrequestid=aeb54fed-7320-4659-bfff-3f0476c1a1f2'

In [ ]:
df

In [672]:
# 데이터 따로 입력해주기

address = []
scores = []
reviews = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(10)
    
    # 주소
    address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
    
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break

    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
    
    # 다음 호텔로 넘어가기
    continue

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.110)
Stacktrace:
	GetHandleVerifier [0x00A06EE3+174339]
	(No symbol) [0x00930A51]
	(No symbol) [0x00646FF6]
	(No symbol) [0x0062EFE7]
	(No symbol) [0x0069B53B]
	(No symbol) [0x006A9E7B]
	(No symbol) [0x00696DA6]
	(No symbol) [0x00671034]
	(No symbol) [0x00671F8D]
	GetHandleVerifier [0x00AA4B1C+820540]
	sqlite3_dbdata_init [0x00B653EE+653550]
	sqlite3_dbdata_init [0x00B64E09+652041]
	sqlite3_dbdata_init [0x00B597CC+605388]
	sqlite3_dbdata_init [0x00B65D9B+656027]
	(No symbol) [0x0093FE6C]
	(No symbol) [0x009383B8]
	(No symbol) [0x009384DD]
	(No symbol) [0x00925818]
	BaseThreadInitThunk [0x773AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77847C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77847C3E+238]


In [616]:
stars = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(2)
    
    # 홈페이지 총점 가져오기
    try :
        stars.append(float(driver.find_element('xpath','//*[@id="property-critical-root"]/div/div[5]/div[2]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/p[1]').text))
    except NoSuchElementException:
        stars.append('')

# 데이터 추가하기
df['rating'] = stars

In [622]:
df.to_csv('./data/Five_Stars_Hotels.csv',index=False)

===================================================================================================================================

4성급 호텔 웹크롤링

In [627]:
df

,name,url,address,rating,review_rating,review
0,스위스 그랜드 호텔 서울 (Swiss Grand Hotel Seoul),https://www.agoda.com/ko-kr/grand-hilton-seoul...,"[서대문구 연희로 353, 서대문, 서울, 대한민국, 03650]",8.0,7.972155,[동생이랑 서울여행 와서 2박 3일 동안 호텔에 머문게 젤 기억에 남을 정도로 편하...
1,JW 메리어트 동대문 스퀘어 서울 (JW Marriott Dongdaemun Squ...,https://www.agoda.com/ko-kr/jw-marriott-dongda...,"[청계천로 279 , 동대문, 서울, 대한민국, 03198]",8.8,8.771654,[1베드룸 이그제큐티브 스위트를 예약하였는데 미리 전화로 호텔측에 연락하여 킹침대 ...
2,그랜드 워커힐 서울 (Grand Walkerhill Seoul),https://www.agoda.com/ko-kr/grand-walkerhill-s...,"[워커힐로 177, 광진, 서울, 대한민국, 04963]",8.8,8.96281,[오래된 곳이러는 인식이 강해 어릴 적 외에 성인이 된 후 스스로 선택해 가 본 적...
3,워커힐 더글라스 하우스 (성인 전용) (Walkerhill Douglas House...,https://www.agoda.com/ko-kr/walkerhill-douglas...,"[서울 광진구 워커힐로 177, 광진, 서울, 대한민국, 04963]",9.1,9.254658,"[워커힐은 제가 6살때 난생처음 야외 수영장으로 방문했던 곳으로, 역사적으로나 제 ..."
4,웨스틴 조선 서울 (The Westin Josun Seoul),https://www.agoda.com/ko-kr/the-westin-chosun-...,"[서울 광진구 워커힐로 177, 광진, 서울, 대한민국, 04963, 서울시 중구 ...",8.9,9.736842,[저희부부는 초등생자녀2명을 데리고 1박2일 서울여행을 계획했습니다. 특히 경복궁을...
5,메이필드 호텔 서울 (Mayfield Hotel Seoul),https://www.agoda.com/ko-kr/mayfield-hotel/hot...,"[방화대로 94, 강서, 서울, 대한민국, 07506]",8.6,8.703364,"[서울 어떤 호텔보다 직원분들이 친절합니다. 체크인부터 수영장이용, 저녁식사, 체크..."
6,이태원에 있는 숙소 (5성급) (5-star hotel in the Itaewon ...,https://www.agoda.com/ko-kr/Agoda-Secret-Deals...,None,,None,None
7,몬드리안 서울 이태원 (Mondrian Seoul Itaewon),https://www.agoda.com/ko-kr/mondrian-seoul-ita...,"[23 Jangmun Ro Yongsan Gu, 이태원, 서울, 대한민국, 04392]",8.7,8.734104,[제일 기본 방으로 예약했는데 큰 킹사이즈침대방으로 업그레이드 해주셨어요! 택시 아...
8,노보텔 앰배서더 서울 강남 (Novotel Ambassador Seoul Gangnam),https://www.agoda.com/ko-kr/novotel-ambassador...,"[서울특별시 강남구 봉은사로 130, 강남, 서울, 대한민국, 06124]",8.3,9.470588,[Parking is good. Room is clean. Subway is con...
9,롯데호텔 월드 (Lotte Hotel World),https://www.agoda.com/ko-kr/lotte-hotel-world/...,"[올림픽로 240 , 송파, 서울, 대한민국, 5554]",8.9,9.135359,[지난 여름에 이어서 두번째 방문이었습니다. 롯데월드와 가깝다는 장점을 배제하고도 ...
